In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
from typing import List, Dict
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from pyvis.network import Network


In [ ]:
load_dotenv('../featuring_network.env')
client = os.getenv('SPOTIFY_CLIENT', 'client')
secret = os.getenv('SPOTIFY_SECRET', 'secret')

#### To Do


- [ ] change edge colour to match genre
- [ ] display genre type in legend
- [ ] fix get pull bug when artist includes '&'
- [ ] investigate issue with missing edges when there is a known collab
- [x] investigate issues with self loops in the graph
- [ ] improve random walk speed...

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client, client_secret=secret))

In [ ]:
def add_song_edge(artist_parent: str, artist_child: str, song_data: Dict, song_id: int, graph: nx.Graph) -> None:
    """
    Args:
        artist_parent - 
        artist_child - 
        song_data - 
        song_id - 
        graph - 
    
    Returns:
    
    """
    graph.add_edge(artist_parent, artist_child,
               name=song_data['name'],
               album_name=song_data['album_name'],
               song_url=song_data['song_url']['spotify'],
               song_id=song_id )


def add_feature_edges(artist_name: str, track_list: Dict, graph: nx.Graph) -> None:
    """
    Args:
        artist_name - 
        track_list - 
        graph - 
    
    Returns: 
    
    """
    print("adding edges")
    
    for track in track_list:
        for collaborator in track_list[track]['features']:
            if collaborator not in graph:
                add_artist_node(collaborator, graph)
            if artist_name != collaborator:
                add_song_edge(artist_name, collaborator, track_list[track],track,  graph)



def add_artist_node(artist_name: str, graph: nx.Graph) -> None:
    """
    Args:
        artist_name - 
        graph - 
    Returns:
    
    """
    print(artist_name)
    artist = get_artist_id(artist_name)    
    if artist is None:
        print('Artist ', artist_name, ' not found!')
        return 
    album_ids = get_artist_album_ids(artist['id'])
    if album_ids is None:
        album_ids = []
    graph.add_node(
        artist['name'], 
        aid=artist['id'],
        albums=list(set(album_ids)),
        genres=artist['genres'],
        followers=artist['followers']['total'],
        href=artist['href'],
        uri=artist['uri'],
        popularity=artist['popularity']
    )


def get_artist_id(name: str) -> str:
    """
    Args:
        name -

    """
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None
    
def get_artist_album_ids(artist_id: str) -> List:
    """
    Args:
        artist_id -
    
    Returns:
    
    """
    album_names = set()
    albums = set()
    results = sp.artist_albums(artist_id, album_type='album')
    for album in results['items']:
        if album['name'] not in album_names:
            albums.add(album['id'])
            album_names.add(album['name'])
    return list(albums)
    
def get_album_track_ids(album_ids: List, artist_id: str) -> Dict:
    """ uses spotipy to get track information for albums in list
    Args:
        album_ids - 
        artist_id - 
        
    Returns:
    
    """
    songs_checker = set()
    songs_w_features = {}
    for album in album_ids:
        track_ids = sp.album_tracks(album)['items']
        for track in track_ids:
            features = [x['name'] for x in track['artists'] if x['uri'] != artist_id]
            # exclude songs w. no features
            # second part of conditional could be an issue if two songs have the same name
            # add song ids to song_checker instead of song names 
            if len(features)>1 and track['id'] not in songs_checker:
                songs_w_features[track['id']] = {}
                songs_w_features[track['id']]['name'] = track['name']
                songs_w_features[track['id']]['features'] = features
                songs_w_features[track['id']]['album_name'] = album
                songs_w_features[track['id']]['song_url'] = track['external_urls']
                songs_checker.add(track['id'])

    return songs_w_features

# TO DO
# random walk graph
# display all features but only expand on one 
# NEED TO INCLUDE EDGES WHEN THERE ARE MULTIPLE FEATURES
# error on 2 chains, dtype, search 
# if artist has no features, end?

def random_walk_graph(artist: str, n_iterations: int) -> nx.Graph:
    """
    Args:
        artist - 
        n_iterations - 
    Returns: 
    
    """
    graph = nx.Graph()
    n_steps = 6
    for i in range(n_steps):
        print("step ", i)
        print("Artist ", artist)
        add_artist_node(artist, graph)
        song_ids = get_album_track_ids(graph.nodes[artist]['albums'], graph.nodes[artist]['aid'])
        add_feature_edges(artist, song_ids, graph)
        # choose value from song_ids, excluding main artist
        all_feats = [x['features'][1:] for x in song_ids.values()]
        if len(all_feats)>0:
            possible_paths = [item for sublist in all_feats for item in sublist]
            artist = np.random.choice(possible_paths, 1)[0]
        else:
            break
    return graph

In [ ]:
graph = nx.Graph()

# graph composed of multiple entries, 
for artist in ['Kendrick Lamar', 'Pusha T', 'Kanye West', 'Drake', 'Future']:

    add_artist_node(artist, graph)
    song_ids = get_album_track_ids(graph.nodes[artist]['albums'], graph.nodes[artist]['aid'])
    add_feature_edges(artist, song_ids, graph)


# print(graph.nodes)
# print(graph.edges)



# need to handle this case 'Rick Ross & Big Sean'?
# issue - kanye west and future collabed in good music, but do not show a connection

In [ ]:
net = Network(notebook=True, height='750px', width='100%', bgcolor='#222222', font_color='white')
net.from_nx(graph)
net.show('graph.html')

In [ ]:
graph = random_walk_graph("Young Thug", 4)

net = Network(notebook=True, height='750px', width='100%', bgcolor='#222222', font_color='white')
net.from_nx(graph)

net.show('graph.html')